In [68]:
H = 4.
c = 10.e3
phi = 30.
alpha = 10.
beta = 10.
delta = 5
gamma = 16.e3
import numpy as np

In [69]:
def slip_length(theta):
    L = H * (1 + np.tan(alpha) * np.tan(beta))/(np.sin(theta) - np.cos(theta)*np.tan(beta))
    return L

In [70]:
def area(theta):
    L = slip_length(theta)
    A = np.array([0,0])
    B = np.array([- H * np.tan(alpha), H])
    C = np.array([L * np.cos(theta), L*np.sin(theta)])
    term1 = (B[0]*C[1] - C[0] * B[1])
    term2 = (A[0]*C[1] - C[0] * A[1])
    term3 = (A[0]*B[1] - B[0] * A[1])
    return np.abs((term1 - term2 + term3))/2.

In [71]:
def cohesive_force(theta):
    return c * slip_length(theta)

In [72]:
def weight(theta):
    return gamma*area(theta)

In [73]:
def Qa(theta):
    term1 = weight(theta) * np.cos(alpha+delta) + cohesive_force(theta) * np.sin(alpha+delta-theta)
    term2 = np.cos(alpha+delta+phi-theta)
    return term1/term2

In [1]:
def Qp(theta):
    term1 = weight(theta) * np.cos(alpha-delta) + cohesive_force(theta) * np.sin(-alpha+delta+theta)
    term2 = np.cos(-alpha+delta+phi+theta)
    return term1/term2

In [74]:
def Eact(theta):
    term1 = weight(theta) * np.sin(theta-phi) - cohesive_force(theta) * np.cos(phi)
    term2 = np.cos(alpha+delta+phi-theta)
    return term1/term2

In [74]:
def Epass(theta):
    term1 = weight(theta) * np.sin(theta+phi) + cohesive_force(theta) * np.cos(phi)
    term2 = np.cos(-alpha+delta+phi+theta)
    return term1/term2

In [75]:
def df_dtheta(f,theta,eps=1.e-4):
    return (f(theta+eps) - f(theta-eps))/(2.*eps)

In [76]:
def residual_a(theta):
    return df_dtheta(Eact,theta)

In [76]:
def residual_p(theta):
    return df_dtheta(Epass,theta)

In [78]:
def find_theta_a(theta_0=np.pi/6, tol=1.e-5):
    theta = theta_0
    iteration_counter = 0
    while (np.abs(residual_a(theta)) > tol and iteration_counter < 100):
        theta -= residual_a(theta) / df_dtheta(residual_a,theta)
        iteration_counter += 1
    return theta, iteration_counter

In [78]:
def find_theta_p(theta_0=np.pi/6, tol=1.e-5):
    theta = theta_0
    iteration_counter = 0
    while (np.abs(residual_p(theta)) > tol and iteration_counter < 100):
        theta -= residual_p(theta) / df_dtheta(residual_p,theta)
        iteration_counter += 1
    return theta, iteration_counter

In [1]:
def theta_ag_norm():
    t1 = np.cos(phi - alpha)
    t2 = np.sin(phi - alpha) + np.sqrt((np.sin(phi + delta)*np.cos(alpha-beta))/(np.sin(phi-beta)*np.cos(alpha+delta)))
    print("For the non-cohesive case the norm gives us %.1f °." %((phi + np.arctan(t1/t2))*180/np.pi))
    return (phi + np.arctan(t1/t2))

In [80]:
from ipywidgets import widgets
from ipywidgets import interact

def plot_interactive_earth_pressure():
    @interact(ic=widgets.IntSlider(min=0, max=50, value=10, step=5, description='$c$ / kN m$^{-2}$:'),
              iphi = widgets.IntSlider(min=20, max=45, value=30, step=5, description='$\\varphi$ / °:'),
              igamma=widgets.IntSlider(min=14, max=25, value=18, step=1, description='$\\gamma$ / kN m$^{-3}$:'),
              iH=widgets.IntSlider(min=1, max=15, value=5, step=1, description='$H$ / m:'),
              ialpha = widgets.IntSlider(min=0, max=15, value=5, step=1, description='$\\alpha$ / °:'),
              ibeta = widgets.IntSlider(min=0, max=15, value=5, step=1, description='$\\beta$ / °:'),
              idelta = widgets.IntSlider(min=0, max=20, value=10, step=2, description='$\\delta_\\mathrm{a}$ / °:'),
             )
    def plot_earth_pressure(ic,iphi,igamma,iH,ialpha,ibeta,idelta):
        #unit conversion
        global c
        global gamma
        global phi
        global alpha
        global beta
        global delta
        global H

        H = iH
        c = ic * 1000.
        gamma = igamma * 1000.
        phi = iphi * np.pi/180.
        alpha = ialpha * np.pi/180.
        beta = ibeta * np.pi/180.
        delta = idelta * np.pi/180.
        #Calculation
        theta_a, it = find_theta_a(theta_ag_norm())

        thetas = np.linspace(np.pi/6,np.pi/2*0.9,100)
        fig, ax = plt.subplots(ncols=2,figsize=(16,8))
        ax[1].plot(thetas*180/np.pi,Eact(thetas)/1e3)
        ax[1].plot(theta_a*180/np.pi,Eact(theta_a)/1e3,marker='d',color='red')
        ax[1].set_xlabel('$\\vartheta$ / °')
        ax[1].set_ylabel('$E_\\mathrm{a}$ / kN m$^{-1}$')
        ax[1].text(theta_a*180/np.pi,Eact(theta_a)/1e3,'$E_\\mathrm{a} = %.2f$ kN m$^{-1}$, $\\vartheta$ = %.1f °' 
                   %(Eact(theta_a)/1e3,theta_a*180/np.pi))
        
        ax[0].set_aspect('equal')
        ax[0].axis('off')
        ##Schnittebene
        #ax[0].plot([0,A_x],[0,A_y],ls='-',color='green',lw=3)
        #ax[0].annotate("$A_\\beta$", xy=(A_x*1.01,A_y*1.01),color='green')
        #Reaktionskraft

        G = weight(theta_a)
        scale = 1./np.abs(G)
        offset = 0.5
        G *= scale
        C = c * slip_length(theta_a) * scale
        Q = Qa(theta_a) * scale
        
        ax[0].annotate("", xy=(offset+0, 0), xytext=(offset+0,G),
                       arrowprops=dict(arrowstyle="-|>",lw=3,color='green'),color='green',xycoords='axes fraction')
        ax[0].annotate(" $\\vec{G}$ ", xy=(offset+0,G/2),color='green',xycoords='axes fraction')
        
        ax[0].annotate("", xy=(offset+C*np.cos(theta_a), C*np.sin(theta_a)), xytext=(offset+0,0),
                       arrowprops=dict(arrowstyle="-|>",lw=3,color='blue'),color='blue',xycoords='axes fraction')
        ax[0].annotate(" $\\vec{C}$ ", xy=(offset+C*np.cos(theta_a)/2, C*np.sin(theta_a)/2),color='blue',xycoords='axes fraction')
        
        ax[0].annotate("", xy=(offset+C*np.cos(theta_a)-Q*np.sin(theta_a-phi), C*np.sin(theta_a)+Q*np.cos(theta_a-phi)),
                       xytext=(offset+C*np.cos(theta_a), C*np.sin(theta_a)), 
                       arrowprops=dict(arrowstyle="-|>",lw=3,color='lightblue'),color='lightblue',xycoords='axes fraction')
        ax[0].annotate(" $\\vec{Q}$ ", xy=(offset+C*np.cos(theta_a), C*np.sin(theta_a)),color='lightblue',xycoords='axes fraction')
        
        ax[0].annotate("",xy=(offset+C*np.cos(theta_a)-Q*np.sin(theta_a-phi)+Eact(theta_a)*scale*np.cos(alpha+delta),
                              C*np.sin(theta_a)+Q*np.cos(theta_a-phi)+Eact(theta_a)*scale*np.sin(alpha+delta)),
                       xytext=(offset+C*np.cos(theta_a)-Q*np.sin(theta_a-phi), C*np.sin(theta_a)+Q*np.cos(theta_a-phi)),
                       arrowprops=dict(arrowstyle="-|>",lw=3,color='red'),color='red',xycoords='axes fraction')
        ax[0].annotate("$\\vec{E}_\\mathrm{a}$ ", xy=(offset+C*np.cos(theta_a)-Q*np.sin(theta_a-phi)+Eact(theta_a)*scale*np.cos(alpha+delta)/2,
                              C*np.sin(theta_a)+Q*np.cos(theta_a-phi)+Eact(theta_a)*scale*np.sin(alpha+delta)/2),color='red',xycoords='axes fraction')
        
        #fig.tight_layout()
        plt.show();

In [ ]:
from ipywidgets import widgets
from ipywidgets import interact

def plot_interactive_earth_pressure_passive():
    @interact(ic=widgets.IntSlider(min=0, max=50, value=10, step=5, description='$c$ / kN m$^{-2}$:'),
              iphi = widgets.IntSlider(min=20, max=45, value=30, step=5, description='$\\varphi$ / °:'),
              igamma=widgets.IntSlider(min=14, max=25, value=18, step=1, description='$\\gamma$ / kN m$^{-3}$:'),
              iH=widgets.IntSlider(min=1, max=15, value=5, step=1, description='$H$ / m:'),
              ialpha = widgets.IntSlider(min=0, max=15, value=5, step=1, description='$\\alpha$ / °:'),
              ibeta = widgets.IntSlider(min=0, max=15, value=5, step=1, description='$\\beta$ / °:'),
              idelta = widgets.IntSlider(min=0, max=20, value=10, step=2, description='$\\delta_\\mathrm{p}$ / °:'),
             )
    def plot_earth_pressure(ic,iphi,igamma,iH,ialpha,ibeta,idelta):
        #unit conversion
        global c
        global gamma
        global phi
        global alpha
        global beta
        global delta
        global H

        H = iH
        c = ic * 1000.
        gamma = igamma * 1000.
        phi = iphi * np.pi/180.
        alpha = ialpha * np.pi/180.
        beta = ibeta * np.pi/180.
        delta = idelta * np.pi/180.
        #Calculation
        theta_p, it = find_theta_p(np.pi/4-np.deg2rad(phi)/2)

        thetas = np.linspace(np.pi/10,np.pi/4,100)
        fig, ax = plt.subplots(ncols=2,figsize=(16,8))
        ax[1].plot(thetas*180/np.pi,Epass(thetas)/1e3)
        ax[1].plot(theta_p*180/np.pi,Epass(theta_p)/1e3,marker='d',color='red')
        ax[1].set_xlabel('$\\vartheta$ / °')
        ax[1].set_ylabel('$E_\\mathrm{p}$ / kN m$^{-1}$')
        ax[1].text(theta_p*180/np.pi,Epass(theta_p)/1e3,'$E_\\mathrm{p} = %.2f$ kN m$^{-1}$, $\\vartheta$ = %.1f °' 
                   %(Epass(theta_p)/1e3,theta_p*180/np.pi))
        
        ax[0].set_aspect('equal')
        ax[0].axis('off')
        ##Schnittebene
        #ax[0].plot([0,A_x],[0,A_y],ls='-',color='green',lw=3)
        #ax[0].annotate("$A_\\beta$", xy=(A_x*1.01,A_y*1.01),color='green')
        #Reaktionskraft

        Q = Qp(theta_p)
        scale = 1./np.abs(Q)
        Q *= scale
        offset = 0.5
        G = weight(theta_p) * scale
        C = -c * slip_length(theta_p) * scale
        
        ax[0].annotate("", xy=(offset+0, 0), xytext=(offset+0,G),
                       arrowprops=dict(arrowstyle="-|>",lw=3,color='green'),color='green',xycoords='axes fraction')
        ax[0].annotate(" $\\vec{G}$ ", xy=(offset+0,G/2),color='green',xycoords='axes fraction')
        
        ax[0].annotate("", xy=(offset+C*np.cos(theta_p), C*np.sin(theta_p)), xytext=(offset+0,0),
                       arrowprops=dict(arrowstyle="-|>",lw=3,color='blue'),color='blue',xycoords='axes fraction')
        ax[0].annotate(" $\\vec{C}$ ", xy=(offset+C*np.cos(theta_p)/2, C*np.sin(theta_p)/2),color='blue',xycoords='axes fraction')
        
        ax[0].annotate("", xy=(offset+C*np.cos(theta_p)-Q*np.sin(theta_p+phi), C*np.sin(theta_p)+Q*np.cos(theta_p+phi)),
                       xytext=(offset+C*np.cos(theta_p), C*np.sin(theta_p)), 
                       arrowprops=dict(arrowstyle="-|>",lw=3,color='lightblue'),color='lightblue',xycoords='axes fraction')
        ax[0].annotate(" $\\vec{Q}$ ", xy=(offset+C*np.cos(theta_p)-Q*np.sin(theta_p+phi)/2, C*np.sin(theta_p)+Q*np.cos(theta_p+phi)/2),color='lightblue',xycoords='axes fraction')
        
        ax[0].annotate("",xy=(offset+C*np.cos(theta_p)-Q*np.sin(theta_p+phi)+Epass(theta_p)*scale*np.cos(alpha-delta),
                              C*np.sin(theta_p)+Q*np.cos(theta_p+phi)+Epass(theta_p)*scale*np.sin(alpha-delta)),
                       xytext=(offset+C*np.cos(theta_p)-Q*np.sin(theta_p+phi), C*np.sin(theta_p)+Q*np.cos(theta_p+phi)),
                       arrowprops=dict(arrowstyle="-|>",lw=3,color='red'),color='red',xycoords='axes fraction')
        ax[0].annotate("$\\vec{E}_\\mathrm{p}$ ", xy=(offset+C*np.cos(theta_p)-Q*np.sin(theta_p+phi)+Epass(theta_p)*scale*np.cos(alpha-delta)/2,
                              C*np.sin(theta_p)+Q*np.cos(theta_p+phi)+Epass(theta_p)*scale*np.sin(alpha-delta)/2),color='red',xycoords='axes fraction')
        
        #fig.tight_layout()
        plt.show();